In [34]:
import msprime
from trails.cutpoints import cutpoints_ABC, cutpoints_AB
import re
from ray import shutdown
# %load_ext rpy2.ipython

In [35]:
import pandas as pd
import numpy as np

n_int_AB = 1
n_int_ABC = 1


mu = 2e-8
g = 25
N_AB = 10_000*2*mu
N_BC = 5_000*2*mu
N_ABC = 10_000*2*mu
N_ref = N_ABC
t_1 = 60_000/g*mu
t_2 = (600_000/g*mu-t_1)
t_3 = (6_000_000/g*mu-t_2)
t_A = t_1
t_B = t_1
t_C = t_1+t_2
r = 1e-8/mu
t_m = t_1-54_000/g*mu
m = 0.05
tot_lst = []
t_A = t_1
t_B = t_1
t_C = t_1+t_2
mu = mu/mu


# t_A = 3.1620008490961066e-05	
# t_B = 2.4e-05	
# t_C = 0.0005055876154267558	
# t_2 = 0.00046274102467880535	
# t_upper = 0.0047843455532931446	
# t_m = 7.776608669717413e-06	
# N_AB = 0.0004616482106878883	
# N_BC = 0.00018440568432470078	
# N_ABC = 0.00042033924549322356	
# r = 0.3893330871941284	
# m = 0.04868512817268997
# mu = 1
# t_C_prime = t_C-t_2
# t_1 = max([t_A, t_B, t_C_prime])
# t_3 = t_upper+cutpoints_ABC(n_int_ABC, 1/N_ABC)[-2]



t_out = t_1+t_2+t_3+2*N_ABC


seed = 2

In [36]:
m*(2/3)*np.exp(-(t_2+t_m)/(N_AB))+(1-m)*(2/3)*np.exp(-(t_2)/(N_BC))

0.08422406774758104

In [37]:
t_m > t_A

False

In [38]:
from trails.optimizer import trans_emiss_calc_introgression

transitions, emissions, starting, hidden_states, observed_states = trans_emiss_calc_introgression(
    t_1, t_1, t_C, t_2, t_upper, t_out, t_m,
    N_AB, N_BC, N_ABC, r, m, n_int_AB, n_int_ABC)

shutdown()

transitions

array([[9.99923348e-01, 2.52477427e-05, 2.52477427e-05, 2.52477427e-05,
        9.08453038e-07],
       [1.34638911e-04, 9.99686824e-01, 8.74990680e-05, 8.73511422e-05,
        3.68710521e-06],
       [1.34638911e-04, 8.74990680e-05, 9.99688924e-01, 8.52511831e-05,
        3.68710521e-06],
       [1.34638911e-04, 8.73511422e-05, 8.52511831e-05, 9.99689072e-01,
        3.68710521e-06],
       [1.28452305e-05, 9.77635393e-06, 9.77635393e-06, 9.77635393e-06,
        9.99957826e-01]])

In [39]:
starting

array([0.62738425, 0.11764828, 0.11764828, 0.11764828, 0.04437049])

In [40]:
pd.DataFrame(transitions, columns=[str(i) for i in hidden_states.values()]).to_csv(f'tables/transitions_{n_int_ABC}.csv', index = False)

In [41]:
n_sites = 1_000_000

demography = msprime.Demography()
demography.add_population(name="A", initial_size=N_AB, default_sampling_time=t_1-t_A)
demography.add_population(name="B", initial_size=N_AB, default_sampling_time=t_1-t_B)
demography.add_population(name="B_anc", initial_size=N_AB, initially_active=False)
demography.add_population(name="C", initial_size=N_BC, default_sampling_time=t_1+t_2-t_C)
demography.add_population(name="D", initial_size=N_AB, default_sampling_time=t_1-t_1)
demography.add_population(name="AB", initial_size=N_AB)
demography.add_population(name="ABC", initial_size=N_ABC)
demography.add_population(name="ABCD", initial_size=N_ABC)
demography.add_admixture(time = t_1-t_m, derived="B", ancestral=["B_anc", "C"], proportions=(1-m, m))
demography.add_population_split(time=t_1, derived=["A", "B_anc"], ancestral="AB")
demography.add_population_split(time=t_1+t_2, derived=["AB", "C"], ancestral="ABC")
demography.add_population_split(time=t_1+t_2+t_3, derived=["ABC", "D"], ancestral="ABCD")

# demography.debug()

ts = msprime.sim_ancestry(
    {"A": 1, "B": 1, "C": 1,
     "D": 1
    },
    demography=demography,
    recombination_rate=r,
    sequence_length=n_sites,
    ploidy=1,
    random_seed=seed
)



In [42]:
# Add mutations
dct = {v: k for k, v in observed_states.items()}

mutated_ts = msprime.sim_mutations(ts, rate=mu, random_seed=seed)

nochange_lst = [dct['AAAA'], dct['CCCC'], dct['TTTT'], dct['GGGG']]
sim_genome = np.random.choice(nochange_lst, n_sites)

mut_lst = []
mut_loc = []
for variant in mutated_ts.variants():
    mut_loc.append(variant.site.position)
    mut_lst.append(''.join([variant.alleles[i] for i in variant.genotypes]))

for i in range(len(mut_loc)):
    sim_genome[int(mut_loc[i])] = dct[mut_lst[i]]

E = sim_genome

In [43]:
# from trails.optimizer import loglik_wrapper

# loglik = loglik_wrapper(transitions, emissions, starting, [E])

# loglik

In [44]:
from trails.optimizer import post_prob_wrapper

postprobs = post_prob_wrapper(transitions, emissions, starting, [E])


In [45]:
postprobs_df = pd.DataFrame(postprobs[0], columns=[str(i) for i in hidden_states.values()])
postprobs_df = postprobs_df[0:1000000]

In [46]:
postprobs_df.to_csv(f'./tables/postprobs_df_{n_int_ABC}.csv', index = False)

In [47]:
postprobs_df

,"(0, 0, 0)","(1, 0, 0)","(2, 0, 0)","(3, 0, 0)","(4, 0, 0)"
0,0.975709,0.008257,0.006603,0.009264,0.000167
1,0.975721,0.008254,0.006599,0.009260,0.000167
2,0.975732,0.008250,0.006594,0.009257,0.000167
3,0.975744,0.008246,0.006590,0.009254,0.000167
4,0.975756,0.008242,0.006585,0.009250,0.000167
...,...,...,...,...,...
999995,0.854632,0.064148,0.030321,0.048860,0.002039
999996,0.854531,0.064181,0.030355,0.048893,0.002040
999997,0.854430,0.064214,0.030388,0.048926,0.002042
999998,0.854329,0.064246,0.030421,0.048959,0.002044


In [48]:
# import demesdraw
# import matplotlib.pyplot as plt

In [49]:
# graph = msprime.Demography.to_demes(demography)
# fig, ax = plt.subplots()  # use plt.rcParams["figure.figsize"]
# demesdraw.tubes(graph, ax=ax, seed=1)
# plt.show()

In [50]:
# ts.draw_svg(y_axis=True)

In [51]:
cut_ABC = t_1+t_2+cutpoints_ABC(n_int_ABC, 1/N_ABC)
cut_AB = t_1+cutpoints_AB(n_int_AB, t_2, 1/N_AB)
cut_AB_2 = np.array([t_1-t_m]+list(cut_AB)[1::])

left_lst = []
right_lst = []
tree_state = []
t_AB_vec = []
t_ABC_vec = []
for t in ts.trees():
    # Append start coordinate
    left_lst.append(t.interval.left)
    # Append end coordinate
    right_lst.append(t.interval.right-1)
    # Get all non-zero coalescent times
    ntimes = [ts.nodes()[n].time for n in t.nodes() if ts.nodes()[n].time not in [0, t_1-t_A, t_1-t_B, t_1-t_C]]
    ntimes = sorted(ntimes)
    # Get time of the first event
    mint = ntimes[0]
    mint2 = ntimes[1]
    # Find topology
    find_re = re.findall("n\d,n\d", t.as_newick(include_branch_lengths=False))[0]
    # Sort species within topology
    find_re = sorted(find_re.split(','))
    # If V0 or V1
    if find_re == ['n0', 'n1']:
        # If the time of the first coalescent is larger than the deepest speciation event
        if mint>=(t_1+t_2):
            state = (1, (mint>cut_ABC).sum()-1, (mint2>cut_ABC).sum()-1)
            # Append V1 state
        else:
            state = (0, (mint>cut_AB).sum()-1, (mint2>cut_ABC).sum()-1)
            # Append V0 state
    # If V2
    elif find_re == ['n0', 'n2']:
        state = (2, (mint>cut_ABC).sum()-1, (mint2>cut_ABC).sum()-1)
    # If V3
    elif find_re == ['n1', 'n2']:
        if mint>=(t_1+t_2):
            state = (3, (mint>cut_ABC).sum()-1, (mint2>cut_ABC).sum()-1)
        else:
            state = (4, (mint>cut_AB_2).sum()-1, (mint2>cut_ABC).sum()-1)
    else:
        state = (5, (mint>cut_ABC).sum()-1, (mint2>cut_ABC).sum()-1)
        print((find_re, (mint>cut_ABC).sum()-1, (mint2>cut_ABC).sum()-1))
    tree_state.append(state)
    t_AB_vec.append(mint)
    t_ABC_vec.append(mint2)

In [52]:
set(tree_state)

{(0, 0, 0), (1, 0, 0), (2, 0, 0), (3, 0, 0), (4, 0, 0)}

In [53]:
df = pd.DataFrame()
df['state'] = [str(i) for i in tree_state]
df['start'] = left_lst
df['end'] = right_lst
df

,state,start,end
0,"(1, 0, 0)",0.0,186.0
1,"(0, 0, 0)",187.0,806.0
2,"(0, 0, 0)",807.0,3624.0
3,"(0, 0, 0)",3625.0,4189.0
4,"(0, 0, 0)",4190.0,5207.0
...,...,...,...
1139,"(0, 0, 0)",994842.0,996364.0
1140,"(0, 0, 0)",996365.0,996787.0
1141,"(0, 0, 0)",996788.0,998000.0
1142,"(0, 0, 0)",998001.0,998715.0


In [54]:
df.to_csv(f'./tables/df_{n_int_ABC}.csv', index = False)
